In [1]:
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, merge
from dotenv import load_dotenv

In [8]:
filepath = '/Users/jasongersing/PycharmProjects/fastApiProject/.env'
load_dotenv(filepath)
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [9]:
db_GB = MongoClient(url)[database][collection_GB]
grape_buyers = DataFrame(
    db_GB.find(projection={
        "_id": False, 
        "profile_id": True, 
        "grapes_seeking": True, 
        "volume_seeking": True, 
    })
)
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])
grape_buyers.rename(columns={"grapes_seeking": "variety", "volume_seeking": "tons"}, inplace=True)
grape_buyers["side"] = "Buyer"
grape_buyers["tons"] = Series([int(num) for num in grape_buyers["tons"]])
grape_buyers.head()

,profile_id,variety,tons,side
0,8n1y23Qyw37t1Y2X,Semillon,45,Buyer
0,8n1y23Qyw37t1Y2X,Granache,45,Buyer
0,8n1y23Qyw37t1Y2X,Pinot gris,45,Buyer
0,8n1y23Qyw37t1Y2X,Viognier,45,Buyer
1,5p0Z81Z5HM1eqL02,Sangiovese,25,Buyer


In [10]:
db_GS = MongoClient(url)[database][collection_GS]
grape_sellers = DataFrame(
    db_GS.find(projection={
        "_id": False, 
        "profile_id": True, 
        "grapes_selling": True, 
        "volume_selling": True,
    })
)
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])
grape_sellers.rename(columns={"grapes_selling": "variety", "volume_selling": "tons"}, inplace=True)
grape_sellers["side"] = "Seller"
grape_sellers["tons"] = Series([int(num) for num in grape_sellers["tons"]])
grape_sellers.head()

,profile_id,variety,tons,side
0,R87s0gF405b4sf0b,Nebbiolo,15,Seller
0,R87s0gF405b4sf0b,Cabernet Sauvignon,15,Seller
0,R87s0gF405b4sf0b,Riesling,15,Seller
1,wbKU48d1y00R774s,Pinot blanc,50,Seller
1,wbKU48d1y00R774s,Petit Verdot,50,Seller


In [11]:
together = merge(grape_buyers, grape_sellers, how='inner', on=['variety'])
together = together.loc[(together.tons_x <= together.tons_y)]
together["tons_difference"] = together["tons_y"] - together["tons_x"]
together.head()

,profile_id_x,variety,tons_x,side_x,profile_id_y,tons_y,side_y,tons_difference
2,8n1y23Qyw37t1Y2X,Semillon,45,Buyer,l83UDWaSB701N183,50,Seller,5
6,0l20pP78UJ81JZD4,Semillon,5,Buyer,3UTk51W7745BZxv1,20,Seller,15
7,0l20pP78UJ81JZD4,Semillon,5,Buyer,2ubo3U4R0LiJ1220,15,Seller,10
8,0l20pP78UJ81JZD4,Semillon,5,Buyer,l83UDWaSB701N183,50,Seller,45
9,0l20pP78UJ81JZD4,Semillon,5,Buyer,FW5uI7D5z3I073M8,30,Seller,25


In [12]:
def return_seller_profile_ids(n_matches: int, profile_id: str):
    buyer_indexes = (together[together["profile_id_x"] == profile_id]).index.tolist()
    seller_profile_id = [together.iloc[num]["profile_id_y"] for num in buyer_indexes]
    return seller_profile_id[:n_matches]

In [14]:
return_seller_profile_ids(3, '8n1y23Qyw37t1Y2X')

['2ubo3U4R0LiJ1220', '428F8dE2IE70Vh2L', 'K76Y770Hgl16y6bR']